In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import wfdb
import ast
import os
from tqdm import tqdm

In [2]:
# PTB-XL ana CSV dosyasını yükle
df = pd.read_csv('d:/ecg/data/raw/ptbxl/ptbxl_database.csv')

# Tüm kayıtları kullan
print(f"Toplam kayıt sayısı: {len(df)}")

Toplam kayıt sayısı: 21837


In [3]:
def load_signal(record_name, data_path='d:/ecg/data/raw/ptbxl/'):
    full_path = os.path.join(data_path, record_name)
    record = wfdb.rdrecord(full_path)
    return record.p_signal

signals = []
labels = []
missing = 0
for idx, row in tqdm(df.iterrows(), total=len(df)):
    try:
        sig = load_signal(row['filename_hr'])
        signals.append(sig)
        labels.append(list(ast.literal_eval(row['scp_codes']).keys()))
    except FileNotFoundError:
        missing += 1
        # print(f"Dosya bulunamadı: {row['filename_hr']}")
print(f"Toplam {missing} kayıt atlandı.")
signals = np.array(signals)

  2%|▏         | 344/21837 [00:06<06:47, 52.74it/s]


KeyboardInterrupt: 

In [4]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(labels)
print("Tüm hastalık türleri:", mlb.classes_)
print("signals shape:", signals.shape)
print("y shape:", y.shape)

Tüm hastalık türleri: ['1AVB' 'ABQRS' 'AFIB' 'AFLT' 'ALMI' 'AMI' 'ANEUR' 'ASMI' 'CLBBB' 'CRBBB'
 'DIG' 'EL' 'HVOLT' 'ILBBB' 'ILMI' 'IMI' 'INVT' 'IRBBB' 'ISCAL' 'ISCAS'
 'ISCIL' 'ISCIN' 'ISCLA' 'ISC_' 'IVCD' 'LAFB' 'LAO/LAE' 'LMI' 'LNGQT'
 'LOWT' 'LPFB' 'LPR' 'LVH' 'LVOLT' 'NDT' 'NORM' 'NST_' 'NT_' 'PAC' 'PACE'
 'PVC' 'QWAVE' 'RAO/RAE' 'RVH' 'SARRH' 'SBRAD' 'SEHYP' 'SR' 'STACH' 'STD_'
 'SVTAC' 'TAB_' 'VCLVH']


AttributeError: 'list' object has no attribute 'shape'

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    signals, y, test_size=0.2, random_state=42
)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

AttributeError: 'list' object has no attribute 'shape'

In [6]:
X_train_t = torch.tensor(X_train, dtype=torch.float32).permute(0,2,1)  # (batch, channels, seq_len)
X_test_t = torch.tensor(X_test, dtype=torch.float32).permute(0,2,1)
y_train_t = torch.tensor(y_train, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32)

train_ds = TensorDataset(X_train_t, y_train_t)
test_ds = TensorDataset(X_test_t, y_test_t)

train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=32)

C:\Users\pasa-\AppData\Local\Temp\ipykernel_15508\3425177757.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  X_train_t = torch.tensor(X_train, dtype=torch.float32).permute(0,2,1)  # (batch, channels, seq_len)


In [8]:
class InceptionBlock1D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.branch1 = nn.Conv1d(in_channels, out_channels, kernel_size=1, padding=0)
        self.branch2 = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
        self.branch3 = nn.Conv1d(in_channels, out_channels, kernel_size=5, padding=2)
        self.branch4 = nn.MaxPool1d(kernel_size=3, stride=1, padding=1)
        self.branch4_conv = nn.Conv1d(in_channels, out_channels, kernel_size=1, padding=0)
        self.bn = nn.BatchNorm1d(out_channels * 4)
        self.relu = nn.ReLU()
    def forward(self, x):
        b1 = self.branch1(x)
        b2 = self.branch2(x)
        b3 = self.branch3(x)
        b4 = self.branch4_conv(self.branch4(x))
        out = torch.cat([b1, b2, b3, b4], dim=1)
        out = self.bn(out)
        return self.relu(out)

class InceptionTime1D(nn.Module):
    def __init__(self, in_channels, n_classes):
        super().__init__()
        self.incept1 = InceptionBlock1D(in_channels, 32)
        self.incept2 = InceptionBlock1D(32*4, 64)
        self.incept3 = InceptionBlock1D(64*4, 128)
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(128*4, n_classes)
    def forward(self, x):
        x = self.incept1(x)
        x = self.incept2(x)
        x = self.incept3(x)
        x = self.global_pool(x).squeeze(-1)
        x = self.dropout(x)
        return torch.sigmoid(self.fc(x))  # Çoklu etiket için sigmoid

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = InceptionTime1D(in_channels=12, n_classes=y.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCELoss()

n_epochs = 20
for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    for xb, yb in train_dl:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {total_loss/len(train_dl):.4f}")

KeyboardInterrupt: 

In [ ]:
model.eval()
all_preds = []
all_targets = []
with torch.no_grad():
    for xb, yb in test_dl:
        xb = xb.to(device)
        preds = model(xb).cpu().numpy()
        all_preds.append(preds)
        all_targets.append(yb.numpy())
all_preds = np.vstack(all_preds)
all_targets = np.vstack(all_targets)

# Her hastalık için threshold 0.5 ile tahmin
y_pred_bin = (all_preds > 0.5).astype(int)
print(classification_report(all_targets, y_pred_bin, target_names=mlb.classes_))

In [ ]:
for idx, disease in enumerate(mlb.classes_):
    tp = np.sum((all_targets[:,idx]==1) & (y_pred_bin[:,idx]==1))
    fn = np.sum((all_targets[:,idx]==1) & (y_pred_bin[:,idx]==0))
    fp = np.sum((all_targets[:,idx]==0) & (y_pred_bin[:,idx]==1))
    print(f"{disease}: TP={tp}, FN={fn}, FP={fp}")

In [ ]:
from collections import Counter
all_labels = [l for sublist in labels for l in sublist]
counter = Counter(all_labels)
plt.figure(figsize=(16,4))
plt.bar(counter.keys(), counter.values())
plt.xticks(rotation=90)
plt.title("Sınıf Dağılımı")
plt.show()